In [5]:
MODEL_VERSION = '7'

In [2]:
# prompt: write a gradio which take image input and image output. Use yolov8 from analytics to do the input image processing. The processed image will be the output.

!pip install ultralytics==8.0.134
!pip install gradio






In [3]:
import os
HOME = os.getcwd()
HOME = '/content'
print(HOME)

from google.colab import drive
GOOGLE_DRIVE = f'{HOME}/drive'
drive.mount(GOOGLE_DRIVE)

/content
Mounted at /content/drive


In [9]:
class Experiment:
  '''
  Google drive path:

  PROJECT_DRIVE = MyDrive/ai/projects/palmtree
  EXPERIMENT_DRIVE = MyDrive/ai/projects/palmtree/experiments
  UPLOAD_DRIVE = MyDrive/ai/projects/palmtree/uploads
  DATASET_DRIVE = MyDrive/ai/projects/palmtree/datasets
  '''
  EPOCHS = 200
  IMGSZ = 640
  MODEL_NAME = "yolov8s"
  EXPERIMENT_HOME = f'{HOME}/experiments/{MODEL_VERSION}'
  DATASET_HOME = f'{HOME}/datasets/palm-tree-detection-{MODEL_VERSION}'
  WEIGHTS_HOME = f'{HOME}/weights/{MODEL_VERSION}'

  PROJECT_DRIVE = f'{GOOGLE_DRIVE}/MyDrive/ai/projects/palmtree'
  EXPERIMENT_DRIVE = f'{PROJECT_DRIVE}/experiments/{MODEL_VERSION}'
  DATASET_DRIVE = f'{PROJECT_DRIVE}/datasets/palm-tree-detection-{MODEL_VERSION}'
  WEIGHTS_DRIVE = f'{PROJECT_DRIVE}/weights/{MODEL_VERSION}'


print(Experiment.EXPERIMENT_HOME)


/content/experiments/7


In [7]:

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.134 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.4/225.8 GB disk)


In [8]:
%cd {HOME}
%mkdir -p {HOME}/weights/
!cp -fpR {Experiment.WEIGHTS_DRIVE} /content/weights/


/content


In [13]:
from ultralytics import YOLO

model = YOLO(f'/content/weights/7/best.pt')
results = model.predict(source='https://media.roboflow.com/notebooks/examples/dog.jpeg', conf=0.25)


Found https://media.roboflow.com/notebooks/examples/dog.jpeg locally at dog.jpeg
image 1/1 /content/dog.jpeg: 640x384 5 healthys, 465.8ms
Speed: 2.6ms preprocess, 465.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)


In [18]:
from PIL import Image


#results[0].boxes.xyxy
#results[0].boxes.conf
#results[0].boxes.cls

#print(results[0])

#for r in results:
#  im_array = r.plot()  # plot a BGR numpy array of predictions
#  im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
#  im.show()



In [ ]:
import gradio as gr
import torch

model = YOLO(f'/content/weights/7/best.pt')

def yolo(im, size=640):
    g = (size / max(im.size))  # gain
    im = im.resize((int(x * g) for x in im.size), Image.ANTIALIAS)  # resize

    results = model.predict(im, conf=0.25)  # inference
    im_array = results[0].plot()  # updates results.imgs with boxes and labels
    imo = Image.fromarray(im_array[..., ::-1])  # RGB PIL image

    return imo

torch.hub.download_url_to_file('https://github.com/ultralytics/yolov5/raw/master/data/images/zidane.jpg', 'zidane.jpg')
torch.hub.download_url_to_file('https://github.com/ultralytics/yolov5/raw/master/data/images/bus.jpg', 'bus.jpg')

inputs = gr.inputs.Image(type='pil', label="Original Image")
outputs = gr.outputs.Image(type="pil", label="Output Image")

title = "YOLOv5"
description = "YOLOv5 Gradio demo for object detection. Upload an image or click an example image to use."
article = "<p style='text-align: center'>YOLOv5 is a family of compound-scaled object detection models trained on the COCO dataset, and includes " \
          "simple functionality for Test Time Augmentation (TTA), model ensembling, hyperparameter evolution, " \
          "and export to ONNX, CoreML and TFLite. <a href='https://github.com/ultralytics/yolov5'>Source code</a> |" \
          "<a href='https://apps.apple.com/app/id1452689527'>iOS App</a> | <a href='https://pytorch.org/hub/ultralytics_yolov5'>PyTorch Hub</a></p>"

examples = [['zidane.jpg'], ['bus.jpg']]
examples = [['test1.jpg']]

gr.Interface(yolo, inputs, outputs, title=title, description=description, article=article, examples=examples, analytics_enabled=False).launch(
    debug=True)

<ipython-input-11-76b274b70c7d>:20: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs = gr.inputs.Image(type='pil', label="Original Image")
<ipython-input-11-76b274b70c7d>:20: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs = gr.inputs.Image(type='pil', label="Original Image")
<ipython-input-11-76b274b70c7d>:21: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs = gr.outputs.Image(type="pil", label="Output Image")


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
